In [8]:
import timeit
import pyarrow
import pandas as pd
import numpy as np
import time
from itertools import compress  

from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

Below we read the data from a compressed parquet file into a pandas df

In [2]:
start_time = time.time()

df = pd.read_parquet("harmonized-table.parquet.gzip")

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 294.7669460773468


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14300584 entries, 0 to 14300583
Columns: 464 entries, accession to trophic_level
dtypes: object(464)
memory usage: 49.4+ GB


In [3]:
start_time = time.time()

df_describe = df.describe(include='all')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 5109.506125926971


In [4]:
df_describe

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,14300583,0,4079113,185928,13196,16138,3162,16333,12891,20305,...,999,33672,4292,2689,29968,204128,49937,1584,26878,5102
unique,14300583,0,616,9008,384,5710,500,2811,571,686,...,109,4446,875,1278,1226,5283,37670,14,1046,51
top,SAMN01127300,NaN,GRU,Unknown,Ustekinumab 45 mg,not collected,20,Missing: Not provided,missing,not applicable,...,0.3,not applicable,15.1 ÃÂµg/L,not applicable,0.25 g,9606,missing,Yes,proband,heterotroph
freq,1,NaN,1300152,20296,2907,428,418,2257,978,1547,...,146,2621,140,169,3927,92901,293,1053,4343,1057


In [5]:
df_describe.to_csv("harmonized-table.parquet_describe.tsv", sep="\t", header=True, index=True)

In [6]:
df_describe.iloc[3,].value_counts()

300      13
105      10
1         8
492       8
689       7
         ..
2388      1
56998     1
1367      1
344       1
621       1
Name: freq, Length: 347, dtype: int64

In [8]:
df_describe.loc[:, df_describe.isna().any()]

,urogenit_disord,douche,menarche,attribute,hrt
count,0,0,0,0,0
unique,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN


a first pass at values corresponding to 'no information', candidates for NA replacement for ML

In [ ]:
missing_labels = ['NaN', 'Unknown', 'not collected', 'Missing: Not provided', 'missing', 'not applicable', 'not provided', '']

In [5]:
df['env_broad_scale'].describe()

count                      1545145
unique                       17586
top       human-associated habitat
freq                        116964
Name: env_broad_scale, dtype: object

In [6]:
df['env_broad_scale'].value_counts()

human-associated habitat    116964
missing                     108895
urban biome                  86064
not applicable               72041
Bos taurus                   25803
                             ...  
AMC2-6                           1
lake environment25               1
ratGE4.4                         1
alpine grassland 116             1
Organism2                        1
Name: env_broad_scale, Length: 17586, dtype: int64

In [7]:
df['env_medium'].value_counts()

feces                     146111
faeces                    103521
soil                       99029
missing                    98177
not applicable             71522
                           ...  
131955                         1
Stool_02-047-4_6_month         1
feces_GE4.4                    1
Gut_St.Denijs_010              1
feces_21SM.w2d                 1
Name: env_medium, Length: 22520, dtype: int64

In [9]:
!pip install sklearn.preprocessing

  Could not find a version that satisfies the requirement sklearn.preprocessing (from versions: )
No matching distribution found for sklearn.preprocessing


In [3]:
laen = preprocessing.LabelEncoder()
df_encode = laen.fit_transform(df)

ValueError: bad input shape (14300584, 464)

In [ ]:
#onehotencoder = OneHotEncoder(categorical_features = [0])

#df_encode = OneHotEncoder().fit_transform(df)

In [ ]:
#df_encode.to_parquet('harmonized-table__onehot.parquet')

In [6]:
#features = df.columns.drop('env_medium')
#type(features)

pandas.core.indexes.base.Index

In [12]:
df_human_index = df['taxonomy_id'].isin(['9606'])

In [15]:
df_human_index = list(compress(range(len(df_human_index )), df_human_index ))
df_human_index

[93,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 115,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 335,
 338,
 339,
 347,
 348,
 355,
 356,
 357,
 358,
 359,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 459,
 460,
 461,
 462

In [16]:
df_nonhuman = df.drop(df_human_index)
                 

In [17]:
df_nonhuman.shape

(7480877, 464)

In [31]:
df_nonhuman.to_parquet('harmonized-table__nonhuman.parquet')

In [20]:
df.env_package.unique()
env_packages = ['soil', 'Soil', 'sediment', 'plant-associated', 'water', 'seawater', 'wastewater', 'fresh water', 'freshwater sediment', 'sea water', 'sterile water', 'Coastal water body (ENVO:02000049)', 'sea water, [ENVO:00002149]', 'soil-associated', 'Sediment', 'biofilm', 'rock', 'plant', 'Water', 'marine sediment (ENVO:00002113)', 'Unknown', 'Sea water', 'cave', 'Deep-sea water', 'MIGS/MIMS/MIMARKS.sediment', 'MIGS/MIMS/MIMARKS.soil', 'MIGS/MIMS/MIMARKS.plant-associated', 'MIGS/MIMS/MIMARKS.water', 'MIGS/MIMS/MIMARKS.plant-associated', 'MIGS/MIMS/MIMARKS.air', 'ENVO:00000016']


array(['missing', None, 'MIENS_16S', 'water', 'host-associated',
       'host associated', 'soil', 'Soil', 'MIMARKS_16S',
       'miscellaneous natural or artificial environment', 'Human-gut',
       'sediment', 'human-gut', 'MIMARKS_water', 'human-associated',
       'MIGS/MIMS/MIMARKS.human-associated',
       'MIGS/MIMS/MIMARKS.host-associated', 'MIGS/MIMS/MIMARKS.sediment',
       'MIGS/MIMS/MIMARKS.wastewater', 'MIGS/MIMS/MIMARKS.miscellaneous',
       'MIGS/MIMS/MIMARKS.soil', 'MIGS/MIMS/MIMARKS.human-oral',
       'MIGS/MIMS/MIMARKS.plant-associated', 'MIGS/MIMS/MIMARKS.water',
       'MIGS/MIMS/MIMARKS.human-gut', 'MIGS/MIMS/MIMARKS.microbial',
       'seawater', 'MIGS/MIMS/MIMARKS.human-skin', 'wastewater/sludge',
       'plant-associated', 'human-oral', 'MIGS/MIMS/MIMARKS.air',
       'microbial mat/biofilm', 'MIGS/MIMS/MIMARKS.human-vaginal',
       'not applicable', 'Host-associated', 'mouse-gut', 'not collected',
       'Not available', 'air', 'wastewater|sludge', 'miscell

In [24]:
df_env_index = df['env_package'].isin(env_packages)
df_env_index

0           False
1           False
2           False
3           False
4           False
            ...  
14300579    False
14300580    False
14300581    False
14300582    False
14300583    False
Name: env_package, Length: 14300584, dtype: bool

In [25]:
df_onlyenv = df[df_env_index]
df_onlyenv.shape

(34313, 464)

In [27]:
df_onlyenv.to_parquet('harmonized-table__envonly.parquet')

In [28]:
start_time = time.time()

df_onlyenv_describe = df_onlyenv.describe(include='all')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

seconds 1.3723068237304688


In [29]:
df_onlyenv_describe

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,34313,0,0,0,0,2130,683,4955,204,196,...,22,0,24,454,14,7033,0,0,0,2
unique,34313,0,0,0,0,638,248,878,8,15,...,16,0,4,367,1,21,0,0,0,2
top,SAMEA104176466,NaN,NaN,NaN,NaN,Missing: Not Provided,0.5,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",...,36.3,NaN,0.53microMolePerLiter,80,10 L,4577,NaN,NaN,NaN,chemolithotroph
freq,1,NaN,NaN,NaN,NaN,276,24,2001,60,70,...,3,NaN,6,46,14,4739,NaN,NaN,NaN,1


In [117]:
df_onlyenv.head()

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
19559,SAMEA1706329,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19560,SAMEA1706311,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19561,SAMEA1706320,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19562,SAMEA1706300,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19563,SAMEA1706316,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [124]:
#df_onlyenv_notNone = df_onlyenv[df_onlyenv != 'None'].index.values
#df_onlyenv_notNone.shape

#df_onlyenv.value_counts()

#df_onlyenv.groupby("None").count()

df_onlyenv.apply(pd.value_counts)

,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
"""This sample (TARA_A100000165) was collected during the Tara Oceans expedition (2009-2013) at station TARA_018 (latitudeN=35.759, longitudeE=14.2574) on date/time=2009-11-02T08:13, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (tbd-tbd micrometres), and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics analysis. This sample has replicate sample(s): TARA_A100000166.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100000220) was collected during the Tara Oceans expedition (2009-2013) at station TARA_019 (latitudeN=34.2116, longitudeE=13.7684) on date/time=2009-11-11T16:24, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001026) was collected during the Tara Oceans expedition (2009-2013) at station TARA_032 (latitudeN=23.36, longitudeE=37.2183) on date/time=2010-01-11T07:21, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis. This sample has replicate sample(s): TARA_A100001027.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001195) was collected during the Tara Oceans expedition (2009-2013) at station TARA_033 (latitudeN=21.9467, longitudeE=38.2517) on date/time=2010-01-13T07:16, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on board with RNA-Later, and stored in liquid nitrogen for later detection of prokaryote nucleic acid sequences by pyrosequencing methods, and for later metagenomics/transcriptomics analysis. This sample has replicate sample(s): TARA_A100001197.""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""This sample (TARA_A100001401) was collected during the Tara Oceans expedition (2009-2013) at station TARA_031 (latitudeN=27.16, longitudeE=27.1433) on date/time=2010-01-09T07:15, using a PUMP (High Volume Peristaltic Pump). The sample material (saline water (ENVO:00002010), including plankton (ENVO:xxxxxxxx)) was collected at a depth of 3-7 m, targeting a surface water layer (ENVO:00002042) in the marine biome (ENVO:00000447). The sample was size-fractionated (0.22-1.6 micrometres), preserved on

In [125]:
df_onlyenv_cp = df_onlyenv
cats = df_onlyenv_cp.columns
df_onlyenv_cp[cats] = df_onlyenv_cp[cats].astype("category")
df_onlyenv_cp[cats].describe()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,accession,urogenit_disord,gap_consent_short_name,phenotype,dose,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,...,porosity,birth_date,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,label,type_strain,family_relationship,trophic_level
count,34313,0,0,0,0,2130,683,4955,204,196,...,22,0,24,454,14,7033,0,0,0,2
unique,34313,0,0,0,0,638,248,878,8,15,...,16,0,4,367,1,21,0,0,0,2
top,SAMN13905284,NaN,NaN,NaN,NaN,Missing: Not Provided,18.64meter,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",...,36.3,NaN,1.6microMolePerLiter,80,10 L,4577,NaN,NaN,NaN,heterotroph
freq,1,NaN,NaN,NaN,NaN,276,24,2001,60,70,...,3,NaN,6,46,14,4739,NaN,NaN,NaN,1


In [ ]:
df_onlyenv_value_counts = df_onlyenv.apply(pd.value_counts, axis=1)

In [30]:
df_onlyenv_describe.to_csv("harmonized-table.parquet_onlyenv_describe.tsv", sep="\t", header=True, index=True)

In [55]:
print(df_onlyenv_describe.loc['top', :].to_numpy())
type(df_onlyenv_describe.loc['top', :].to_numpy())

['SAMEA104176466' nan nan nan nan 'Missing: Not Provided' '0.5'
 'Missing: Not provided' '02/01/2011-12/01/2011'
 'Anti bird net applied, plants removed in buckets before sampling because of difficulty in access'
 'lifeguard preservation solution' nan nan nan
 'Palacios C, Zettler E, Amils R, Amaral-Zettler L (2008) Contrasting Microbial Community Assembly Hypotheses: A Reconciling Tale from the Rio Tinto. PLoS ONE 3(12): e3853. doi:10.1371/journal.pone.0003853'
 nan '67 kg N ha-1' nan nan nan nan '0.0' 'not applicable'
 '-1 degree Celsius' 'Missing: Not provide' nan nan nan 'Healthy' nan nan
 nan nan nan nan nan nan 'silt' 'Missing: Not provided' nan nan
 'Missing: Not provided' '8.9' nan nan nan nan '457.904'
 'Missing: Not provided' nan nan nan nan nan 'not applicable' nan nan nan
 nan 'Nipponbare' nan nan 'ENVO:cropland biome' nan nan
 '0.05 microgram per liter' nan nan nan 'multiple corer' nan nan nan nan
 '1,485,358 bp' '14_OA11.15' nan nan nan nan nan nan
 'Missing: Not Provided

numpy.ndarray

In [62]:
#index = df_onlyenv_describe['top'].index[df_onlyenv_describe['top'].apply(np.isnan)]
df_onlyenv_index_notnan = np.where(df_onlyenv_describe.loc['top', :].notnull())[0]#np.where(df_onlyenv_describe.loc['top', :].isnan())
#index = list(np.where(df_onlyenv_describe.loc['top', :].isnan()[0]))

df_onlyenv_index_notnan

array([  0,   5,   6,   7,   8,   9,  10,  14,  16,  21,  22,  23,  24,
        28,  37,  38,  41,  42,  47,  48,  54,  59,  62,  65,  69,  74,
        75,  82,  84,  90,  91,  92, 101, 103, 105, 106, 110, 115, 116,
       120, 123, 128, 130, 132, 133, 136, 139, 140, 141, 146, 148, 154,
       155, 157, 161, 162, 166, 167, 168, 169, 171, 173, 174, 177, 178,
       181, 182, 183, 187, 189, 199, 201, 202, 204, 205, 206, 207, 208,
       215, 216, 217, 224, 228, 232, 237, 238, 244, 245, 247, 254, 265,
       270, 278, 284, 286, 287, 288, 289, 291, 295, 296, 297, 298, 301,
       302, 308, 310, 311, 313, 315, 316, 318, 320, 321, 322, 323, 324,
       328, 329, 331, 336, 340, 343, 347, 350, 356, 361, 364, 366, 367,
       371, 373, 375, 376, 381, 384, 389, 391, 392, 395, 396, 398, 400,
       401, 403, 406, 408, 414, 418, 424, 425, 426, 428, 430, 431, 435,
       437, 438, 440, 442, 444, 446, 447, 454, 456, 457, 458, 459, 463])

In [70]:
df_onlyenv_nona = df_onlyenv[df_onlyenv.columns[df_onlyenv_index_notnan]]

#gets opposite, so nans
#df_onlyenv_nona = df_onlyenv.drop(df_onlyenv.columns[df_onlyenv_index_notnan], axis=1)


In [71]:
df_onlyenv_nona_describe = df_onlyenv_nona.describe(include='all')
df_onlyenv_nona_describe

,accession,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,samp_mat_process,heavy_metals_meth,agrochem_addition,nitro,...,host_tissue_sampled,samp_store_loc,petroleum_hydrocarb,extreme_salinity,porosity,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,trophic_level
count,34313,2130,683,4955,204,196,4767,27,411,55,...,325,334,5,23,22,24,454,14,7033,2
unique,34313,638,248,878,8,15,13,1,3,43,...,4,2,4,11,16,4,367,1,21,2
top,SAMEA104176466,Missing: Not Provided,0.5,Missing: Not provided,02/01/2011-12/01/2011,"Anti bird net applied, plants removed in bucke...",lifeguard preservation solution,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",67 kg N ha-1,0.0,...,not applicable,"DRI, Murray lab",1.0 g/kg soil dw,95,36.3,0.53microMolePerLiter,80,10 L,4577,chemolithotroph
freq,1,276,24,2001,60,70,3139,27,355,9,...,310,328,2,6,3,6,46,14,4739,1


In [116]:
df_onlyenv_nona.head()

,accession,tot_nitro,tot_depth_water_col,nitrate,samp_store_dur,perturbation,samp_mat_process,heavy_metals_meth,agrochem_addition,nitro,...,host_tissue_sampled,samp_store_loc,petroleum_hydrocarb,extreme_salinity,porosity,tot_phosp,water_content,samp_vol_we_dna_ext,host_taxid,trophic_level
19559,SAMEA1706329,None,None,None,None,None,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,...,None,None,None,None,None,None,None,None,None,None
19560,SAMEA1706311,None,None,None,None,None,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,...,None,None,None,None,None,None,None,None,None,None
19561,SAMEA1706320,None,None,None,None,None,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,...,None,None,None,None,None,None,None,None,None,None
19562,SAMEA1706300,None,None,None,None,None,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,...,None,None,None,None,None,None,None,None,None,None
19563,SAMEA1706316,None,None,None,None,None,Sterivex filtration,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,...,None,None,None,None,None,None,None,None,None,None


In [79]:
env_series = df_onlyenv_nona['env_package']
print(type(env_series))
env_series

<class 'pandas.core.series.Series'>


19559       water
19560       water
19561       water
19562       water
19563       water
            ...  
14275236     soil
14275237     soil
14275238     soil
14275239     soil
14275240     soil
Name: env_package, Length: 34313, dtype: object

In [74]:
df_onlyenv_nona_num_index = df_onlyenv_nona_describe.loc['top',:].str.isnumeric()
df_onlyenv_nona_num_index

accession              False
tot_nitro              False
tot_depth_water_col    False
nitrate                False
samp_store_dur         False
                       ...  
tot_phosp              False
water_content           True
samp_vol_we_dna_ext    False
host_taxid              True
trophic_level          False
Name: top, Length: 169, dtype: bool

In [75]:
df_onlyenv_nona_num = df_onlyenv_nona[df_onlyenv_nona.columns[df_onlyenv_nona_num_index]]



In [76]:
df_onlyenv_nona_num_describe = df_onlyenv_nona_num.describe(include='all')
df_onlyenv_nona_num_describe

,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
count,49,5781,34313,13,183,12,172,52,538,34313,1257,23,454,7033
unique,1,2,79,2,7,1,150,14,12,34313,121,11,367,21
top,8,0,449393,25818237,2004,9,187,5338,316748,15443275,975,95,80,4577
freq,49,5452,6029,12,150,12,3,6,328,1,708,6,46,4739


In [86]:
df_onlyenv_nona_num.where(df_onlyenv_nona_num == '0 m').describe()

,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
count,0,329,0,0,0,0,0,0,0,0,0,0,0,0
unique,0,1,0,0,0,0,0,0,0,0,0,0,0,0
top,NaN,0 m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
#df_onlyenv_nona_num_str_row_index = df_onlyenv_nona_num.applymap(np.isreal).all(1)

In [93]:
#df_onlyenv_nona_num_str_row_index.describe()

count     34313
unique        1
top       False
freq      34313
dtype: object

In [89]:
#df_onlyenv_nona_num_nostr = df_onlyenv_nona_num[df_onlyenv_nona_num_str_row_index]

In [115]:
df_onlyenv_nona_num.head()

,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
19559,None,None,449393,None,None,None,None,None,None,26206,None,None,None,None
19560,None,None,449393,None,None,None,None,None,None,26207,None,None,None,None
19561,None,None,449393,None,None,None,None,None,None,26208,None,None,None,None
19562,None,None,449393,None,None,None,None,None,None,26209,None,None,None,None
19563,None,None,449393,None,None,None,None,None,None,26210,None,None,None,None


In [101]:
#df_onlyenv_nona_num_nostr = pd.to_numeric(df_onlyenv_nona_num_nostr)
df_onlyenv_nona_num_nostr = df_onlyenv_nona_num.apply(pd.to_numeric, errors="coerce")
#df_onlyenv_nona_num_nostr = df_onlyenv_nona_num.astype(int)

In [105]:
df_onlyenv_nona_num_nostr.head()

,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
19559,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26206,NaN,NaN,NaN,NaN
19560,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26207,NaN,NaN,NaN,NaN
19561,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26208,NaN,NaN,NaN,NaN
19562,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26209,NaN,NaN,NaN,NaN
19563,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26210,NaN,NaN,NaN,NaN


In [102]:
print(df_onlyenv_nona_num_nostr.shape)
df_onlyenv_nona_num_nostr.describe(include='all')

(34313, 14)


,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
count,49.0,5452.0,3.431300e+04,12.0,150.0,12.0,167.000000,52.000000,538.000000,3.431300e+04,1255.000000,23.000000,454.000000,6.888000e+03
mean,8.0,0.0,6.432479e+05,25818237.0,2004.0,9.0,122.106381,26950.969231,323589.141264,8.303156e+06,984.823363,127.086957,43.035549,1.787051e+05
std,0.0,0.0,3.966880e+05,0.0,0.0,0.0,245.912228,21088.525193,83046.636894,3.707262e+06,557.667035,108.261012,39.858090,3.904299e+05
min,8.0,0.0,6.890000e+02,25818237.0,2004.0,9.0,0.453798,4900.000000,167137.000000,2.620600e+04,3.160000,30.000000,7.859208,4.530000e+03
25%,8.0,0.0,4.106580e+05,25818237.0,2004.0,9.0,5.701232,12300.000000,316748.000000,5.602731e+06,870.000000,40.000000,31.872500,4.577000e+03
50%,8.0,0.0,4.127550e+05,25818237.0,2004.0,9.0,24.659574,23400.000000,316748.000000,6.807713e+06,975.000000,95.000000,36.945000,4.577000e+03
75%,8.0,0.0,9.399280e+05,25818237.0,2004.0,9.0,49.461738,30000.000000,316748.000000,9.908403e+06,975.000000,190.000000,43.175000,4.577000e+03
max,8.0,0.0,2.607662e+06,25818237.0,2004.0,9.0,1230.000000,85422.800000,602715.000000,1.573708e+07,6200.000000,332.000000,826.000000,1.671699e+06


In [104]:
df_onlyenv_nona_num_nostr.isnull()

,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
19559,True,True,False,True,True,True,True,True,True,False,True,True,True,True
19560,True,True,False,True,True,True,True,True,True,False,True,True,True,True
19561,True,True,False,True,True,True,True,True,True,False,True,True,True,True
19562,True,True,False,True,True,True,True,True,True,False,True,True,True,True
19563,True,True,False,True,True,True,True,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275236,True,True,False,True,True,True,True,True,True,False,True,True,True,True
14275237,True,True,False,True,True,True,True,True,True,False,True,True,True,True
14275238,True,True,False,True,True,True,True,True,True,False,True,True,True,True
14275239,True,True,False,True,True,True,True,True,True,False,True,True,True,True


In [110]:
df_onlyenv_nona_num_nostr_nan_index = df_onlyenv_nona_num_nostr.isnull()
df_onlyenv_nona_num_nostr_nan_rows_index = df_onlyenv_nona_num_nostr_nan_index.any(axis=1)
print(df_onlyenv_nona_num_nostr_nan_rows_index)
df_onlyenv_nona_num_nostr_nan_rows_index_rows = df_onlyenv_nona_num_nostr[df_onlyenv_nona_num_nostr_nan_rows_index]
df_onlyenv_nona_num_nostr_nan_rows_index_rows

19559       True
19560       True
19561       True
19562       True
19563       True
            ... 
14275236    True
14275237    True
14275238    True
14275239    True
14275240    True
Length: 34313, dtype: bool


,samp_salinity,altitude,taxonomy_id,ref_biomaterial,fire,pool_dna_extracts,microbial_biomass,light_intensity,entrez_value,biosample_id,annual_season_precpt,extreme_salinity,water_content,host_taxid
19559,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26206,NaN,NaN,NaN,NaN
19560,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26207,NaN,NaN,NaN,NaN
19561,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26208,NaN,NaN,NaN,NaN
19562,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26209,NaN,NaN,NaN,NaN
19563,NaN,NaN,449393,NaN,NaN,NaN,NaN,NaN,NaN,26210,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275236,NaN,NaN,256318,NaN,NaN,NaN,NaN,NaN,NaN,15737079,NaN,NaN,NaN,NaN
14275237,NaN,NaN,256318,NaN,NaN,NaN,NaN,NaN,NaN,15737080,NaN,NaN,NaN,NaN
14275238,NaN,NaN,256318,NaN,NaN,NaN,NaN,NaN,NaN,15737081,NaN,NaN,NaN,NaN
14275239,NaN,NaN,256318,NaN,NaN,NaN,NaN,NaN,NaN,15737082,NaN,NaN,NaN,NaN


In [114]:
df_onlyenv_nona_num_nostr_nans = df_onlyenv_nona_num_nostr.isnull().sum(axis=1)
df_onlyenv_nona_num_nostr_nans.describe()

count    34313.000000
mean        11.561332
std          0.608845
min          9.000000
25%         11.000000
50%         12.000000
75%         12.000000
max         12.000000
dtype: float64

In [103]:
start_time = time.time()

features = []

y= env_series#df['env_medium']
X= df_onlyenv_nona_num_nostr #df[features]

###TODO: increase splits
dtree=DecisionTreeClassifier(min_samples_split=20, random_state=9999)
#df_trim = df.drop('env_medium', axis=1) 
dtree.fit(X,y)
print('Decision Tree Classifer Created')

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
visualize_tree(dt, features)

In [ ]:
start_time = time.time()

dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data, feature_names=labels,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

exec_time = (time.time() - start_time)
print('seconds ' + str(exec_time))